In [27]:
import re
import json
from typing import List, Tuple

import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from transformers import DistilBertTokenizer, DistilBertModel
import pandas as pd
import numpy as np

import seaborn as sns

# Data

In [3]:
with open("../data/dev.json") as json_file:
    data = json.load(json_file)

In [86]:
print(data[0]['data']['text'])

PETITIONER:
THE COMMISSIONER OF INCOME-TAXNEW DELHI

Vs.

RESPONDENT:
M/s. CHUNI LAL MOONGA RAM

DATE OF JUDGMENT:
05/05/1961

BENCH:
DAS, S.K.
BENCH:
DAS, S.K.
HIDAYATULLAH, M.
SHAH, J.C.

CITATION:            1962 SCR (2) 823
 1962 AIR 1272

ACT:
Excess Profits Tax-Income-Assesseecarrying on business in
taxable territory-Losses incurred in transactionsin non-
taxable territory-If allowable in computing income-Excess
Profits Tax Act, 1940 (15 of 1940), s. 5.

HEADNOTE:
During the assessment year 1946-47, the assessee was
carrying on speculative business in bullion at Delhi. It
entered into transactions in the nature of forward
transactions with parties at Bhatinda (in the Patiala State
outside the taxable territories of British India) in which
it suffered losses. The assessee claimed deduction of these
losses in the computation of its income.
Held, that the losses incurred in Bhatinda could not be
taken into account in computing the income of the assessee
in British India. Under the t

In [5]:
data[0].keys()

dict_keys(['id', 'annotations', 'data', 'meta'])

In [5]:
# doc = data[0]['data']['text']
# used = doc[:doc.rindex("ACT:")]
# doc = doc[doc.rindex("ACT:"):]
# print(used)
# print("-----------------------------")
# print(doc)

In [6]:
# def extract_sentences(text, start_text):
#     pattern = re.compile(rf'\b{re.escape(start_text)}.*?[.!?](?=\s|$)', re.IGNORECASE | re.DOTALL)
#     matches = re.findall(pattern, text)
#     return matches

# # Example usage:
# # text_to_search = "This is a sample text. Starting sentence with given text. Another sentence with the given text. Not starting with it."

# start_text = "CITATION"
# result = extract_sentences(doc, start_text)

In [6]:
len(data[0]['annotations'])

1

In [7]:
data[0]['annotations'][0].keys()

dict_keys(['result'])

In [8]:
len(data[0]['annotations'][0]['result'])

96

# Tokenizing

In [50]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# print(tokenizer)

### Extract info from dictionary

In [10]:
sent_id = []
sent_start = []
sent_end = []
train_sents = []
target = []
for entry in data:
    _ids, _start, _end, _text, _labels = [], [], [], [], []
    # print(entry)
    # print(entry['annotations'][0]['result'])
    # for x , y in [(data_point['text'], data_point['labels']) for data_point in entry['annotations'][0]['result']['value']]:
    for data_point in entry['annotations'][0]['result']:
        _ids.append(data_point['id'])
        _start.append(data_point['value']['start'])
        _end.append(data_point['value']['end'])
        _text.append(data_point['value']['text'])
        _labels.append(data_point['value']['labels'])
    
    sent_id.append(_ids)
    sent_start.append(_start)
    sent_end.append(_end)
    train_sents.append(_text)
    target.append(_labels)

### Tokenize sentences and add CLS and SEP tokens

In [28]:
# try:
#     doc_lens = []
#     for doc_index, doc in enumerate(train_sents):
#         sent_lens = []
#         for sent_index, sentence in enumerate(doc):
#             train_sents[doc_index][sent_index] = [tokenizer.cls_token] + tokenizer.tokenize(sentence) + [tokenizer.sep_token]
#             sent_lens.append(len(train_sents[doc_index][sent_index]))    
#         doc_lens.append(sent_lens)
    
# except TypeError:
#     print("Sentences already tokenized. Check again")

In [29]:
# print(f"Number of documents : {len(train_sents)}")

In [115]:
# df = pd.DataFrame()
# df['lengths'] = doc_lens 
# df['max_length'] = [np.max(i) for i in doc_lens]  

NameError: name 'doc_lens' is not defined

In [31]:
# df.head()

In [32]:
# df.iloc[0]

In [33]:
# max_length = 0
# for index, row in df.iterrows():
#     sns.histplot(row['lengths'],bins=15)
#     if row['max_length'] > max_length:
#         max_length = row['max_length']
        
# print(max_length)

### Padding the sentences with PAD token

In [34]:
# # MAX_LENGTH = 330
# # attention_matrix = np.zeros(train_sents)
# for doc_index, doc in enumerate(train_sents):
#     MAX_LENGTH = df.iloc[doc_index]['max_length'] + 1
#     for sent_index, sentence in enumerate(doc):
#         train_sents[doc_index][sent_index] = train_sents[doc_index][sent_index] + [tokenizer.pad_token for _ in range(MAX_LENGTH - len(train_sents[doc_index][sent_index]))]
#         # for token in train_sents[doc_index][sent_index]:
#         # attention_matrix[doc_index][sent_index] = [1 if token != tokenizer.pad_token else 0 for token in train_sents[doc_index][sent_index]]

### Creating attention matrix for each document

In [35]:
# ids = []
# segs = []
# for doc_index, doc in enumerate(train_sents):
#     globals()[f"attention_mask_{doc_index}"] = np.zeros_like(doc,dtype='int')
#     sent_id = []
#     sent_segs = []
#     for sent_index,sentence in enumerate(doc):
#         sent_id.append(tokenizer.convert_tokens_to_ids(sentence))
#         sent_segs.append([0 for _ in range(len(sentence))])
#         for i in range(len(sentence)):
#             if sentence[i] != tokenizer.pad_token:
#                 globals()[f'attention_mask_{doc_index}'][sent_index][i] = 1 
#     ids.append(sent_id)
#     segs.append(sent_segs)

In [36]:
# len(ids)
# # len(segs)

In [37]:
# for i in range(len(train_sents)):
#     print(f'Attention Mask for document {i+1} : {globals()[f"attention_mask_{i}"].shape}')

# Extracting sentence embeddings

In [38]:
# model = BertModel.from_pretrained("bert-base-uncased")

In [39]:
# for i in range(len(train_sents)):
#     globals()[f'ids_tensor_{i}'] = torch.LongTensor(ids[i]).unsqueeze(0)
#     globals()[f'attention_mask_{i}'] = torch.from_numpy(globals()[f'attention_mask_{i}']).unsqueeze(0)
#     globals()[f'seg_tokens_{i}'] = torch.LongTensor(segs[i]).unsqueeze(0)

In [40]:
# type(attention_mask_0)

In [41]:
# type(ids_tensor_0)

In [42]:
# type(seg_tokens_0)

In [43]:
# globals()[f'ids_tensor_{0}'][:,0,:]

In [109]:
# for doc_index, doc in enumerate(train_sents):
#     last_hidden_state = []
#     pooler_output = []
#     for sent_index in range(len(doc)):
#         # if doc_index == 0:
#             # print(sent_index)
#         output = model(globals()[f'ids_tensor_{doc_index}'][:,sent_index,:],
#                         attention_mask = globals()[f'attention_mask_{doc_index}'][:,sent_index,:],
#                         token_type_ids = globals()[f'seg_tokens_{doc_index}'][:,sent_index,:])
#         last_hidden_state.append(output[0])
#         pooler_output.append(output[1])
#     globals()[f'last_hidden_state_{doc_index}'], globals()[f'pooler_output_{doc_index}'] = last_hidden_state, pooler_output

: 

In [44]:
# model(ids_tensor_0[:,0,:],attention_mask_0[:,0,:],seg_tokens_0[:,0,:]).pooler_output.size()

In [45]:
# sentence_embeddings = []
# for i in range(15):
#     emb = model(ids_tensor_0[:,i,:],attention_mask_0[:,i,:],seg_tokens_0[:,i,:]).pooler_output
#     sentence_embeddings.append(emb.squeeze(0))

In [46]:
# sentence_embeddings[0]

 - Create function that given a sentence produces id, attention mask and seg
 - Create a function that uses those 3 as input and produces embeddings on the fly

# Tom's work

In [47]:
# from transformers import BertTokenizer, BertModel
# import torch

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

# import json

# file_path = '../data/train.json'

# with open(file_path, 'r') as file:
#     data = json.load(file)
    
#     for key, value in data[0].items():
        
#         if key == 'annotations':
#             for element in value:
#                 for key2, value2 in element.items():
#                     for element2 in value2:
#                         for key3, value3 in element2.items():
#                             if isinstance(value3, dict):
#                                 for key4, value4 in value3.items():
#                                     if key4 == "text":
#                                         print(value4)
                                        
#                                     inputs = tokenizer(key4, return_tensors="pt", truncation=True, padding=True)
#                                     outputs = model(**inputs)

#                                     bert_embedding = outputs.last_hidden_state[:, 0, :]


#                                     bert_embedding_np = bert_embedding.detach().numpy()

                                    
#                                     print(bert_embedding_np.shape)

# Simple CNN implementation

In [9]:
print(f"Number of documents for training : {len(data)}")

Number of documents for training : 30


In [10]:
doc1_train_data = data[0]['annotations'][0]['result']
type(doc1_train_data)

list

In [15]:
labels = []
for entry in doc1_train_data:
    labels.append(entry['value']['labels'][0])

In [26]:
change_count = 0
for index, label in enumerate(labels):
    try:
        if label != labels[index-1]:
            print(index, label, labels[index-1])
            change_count += 1
    except IndexError:
        continue

0 PREAMBLE RPC
10 ANALYSIS PREAMBLE
11 NONE ANALYSIS
18 FAC NONE
47 RLC FAC
50 FAC RLC
51 ARG_PETITIONER FAC
53 ANALYSIS ARG_PETITIONER
68 STA ANALYSIS
70 PRE_RELIED STA
73 ANALYSIS PRE_RELIED
91 RPC ANALYSIS


In [25]:
print(f"Percentage changes from sentence to sentence : {change_count / len(labels) * 100}%")

Percentage changes from sentence to sentence : 12.5%


In [34]:
def sent2tensors(sentence: str, MAX_LEN = None) -> dict:
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    # print(tokenizer.tokenize(sentence))
    if MAX_LEN is None:
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    else:
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding='max_length',max_length = MAX_LEN)
    return inputs

def sent2embeddings(sentence: List, MAX_LEN = None) -> torch.TensorType:
    model = BertModel.from_pretrained('bert-base-uncased')
    
    inputs = sent2tensors(sentence,MAX_LEN)
    emb = model(**inputs).pooler_output
    
    return emb

In [81]:
class SimpleCNN1(nn.Module):
    def __init__(self, input_channels, output_classes):
        super(SimpleCNN1, self).__init__()

        # Convolutional layer
        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=1, kernel_size=(1,5), stride=(1,5))
        
        # ReLU activation
        self.relu = nn.ReLU()

        # Max pooling layer
        self.max_pool = nn.MaxPool2d(kernel_size = (1,2), stride = (1,2))

        # Fully connected layer
        # self.fc1 = nn.Linear(32 * 383, output_classes)
        
    def forward(self, x):
        # Forward pass through the layers
        x = self.conv1(x)
        print(x.size())
        x = self.relu(x)
        x = self.max_pool(x)
        print(x.size())
        # x = x.view(x.size(0), -1)  # Flatten the output for the fully connected layer
        # x = self.fc1(x)
        return x
    
class SimpleCNN2(nn.Module):
    def __init__(self, input_channels, output_classes):
        super(SimpleCNN2, self).__init__()

        # Convolutional layer
        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=1, kernel_size=(2,5), stride=(1,5))
        
        # ReLU activation
        self.relu = nn.ReLU()

        # Max pooling layer
        self.max_pool = nn.MaxPool2d(kernel_size = (1,2), stride = (1,2))

        # Fully connected layer
        # self.fc1 = nn.Linear(32 * 383, output_classes)
        
    def forward(self, x):
        # Forward pass through the layers
        x = self.conv1(x)
        print(x.size())
        x = self.relu(x)
        x = self.max_pool(x)
        print(x.size())
        # x = x.view(x.size(0), -1)  # Flatten the output for the fully connected layer
        # x = self.fc1(x)
        return x

In [82]:
CNN1 = SimpleCNN1(1,8)
CNN2 = SimpleCNN2(1,8)

In [84]:
# test_sentence = doc1_train_data[0]['value']['text']
# emb_test_sent = sent2embeddings(test_sentence)
# emb_test_sent.size()
a = torch.Tensor()
for i in range(10):
    test_sentence = doc1_train_data[i]['value']['text']
    emb_test_sent = sent2embeddings(test_sentence)
    a = torch.cat((a,emb_test_sent),dim=0)
    
a.size()

torch.Size([10, 768])

In [85]:
cnn_out1 = CNN1.forward(a.unsqueeze(0))
cnn_out2 = CNN2.forward(a.unsqueeze(0))

torch.Size([1, 10, 153])
torch.Size([1, 10, 76])
torch.Size([1, 9, 153])
torch.Size([1, 9, 76])


In [51]:
cnn_out.size()

torch.Size([1, 1, 76])

In [61]:
cnn_out

tensor([[[0.2111, 0.2538, 0.0000, 0.3890, 0.0000, 0.0210, 0.0000, 0.0000,
          0.0000, 0.2214, 0.0000, 0.0000, 0.1427, 0.0000, 0.0000, 0.0336,
          0.2356, 0.0067, 0.5807, 0.1057, 0.0131, 0.3784, 0.3886, 0.0000,
          0.3275, 0.0000, 0.3735, 0.0000, 0.2242, 0.2170, 0.0000, 0.4937,
          0.0825, 0.0524, 0.2628, 0.4775, 0.2935, 0.5830, 0.0000, 0.0257,
          0.2118, 0.1271, 0.4794, 0.1105, 0.4283, 0.1057, 0.0099, 0.5743,
          0.0000, 0.1275, 0.4188, 0.4332, 0.0687, 0.1553, 0.1363, 0.0000,
          0.1185, 0.0000, 0.0000, 0.4724, 0.2046, 0.0000, 0.1247, 0.2693,
          0.5646, 0.0207, 0.0000, 0.0000, 0.0000, 0.6681, 0.2004, 0.2905,
          0.1377, 0.5044, 0.6974, 0.4104]]],
       grad_fn=<MaxPool2DWithIndicesBackward0>)

In [128]:
# def extract_word_embeddings(sentence,MAX_LEN = None):
#     # Load pre-trained DistilBERT model and tokenizer
#     model_name = 'distilbert-base-uncased'
#     model = DistilBertModel.from_pretrained(model_name)
#     tokenizer = DistilBertTokenizer.from_pretrained(model_name)
#     # print(tokenizer.tokenize(sentence))

#     # Tokenize the input sentence
#     tokens = tokenizer(sentence, return_tensors='pt', padding='max_length', truncation=True, max_length = MAX_LEN, add_special_tokens=True)

#     # Obtain the output embeddings from the model
#     with torch.no_grad():
#         outputs = model(**tokens)

#     # Extract the embeddings for each token
#     embeddings = outputs.last_hidden_state

#     return embeddings

In [124]:
# max([len(tokenizer.tokenize(sent)) for sent in train_sents[0]])

In [152]:
# MAX_LENGTH = max([len(tokenizer.tokenize(sent)) for sent in train_sents[0]])
# result = torch.Tensor()
# for sentence in train_sents[0]:
#     embedding = sent2embeddings(sentence,MAX_LEN = MAX_LENGTH)
#     result = torch.cat((result,embedding),dim=0)
# sentence1 = train_sents[0][0]
# sentence2 = train_sents[0][1]
# MAX_LENGTH = 30
# print(sentence1)
# print(sentence2)
# emb1 = sent2embeddings(sentence1,MAX_LEN = MAX_LENGTH)
# emb2 = sent2embeddings(sentence2,MAX_LEN = MAX_LENGTH)
# print(emb1.size())
# print(emb2.size())

['petition', '##er', ':', 'the', 'commissioner', 'of', 'income', '-', 'tax', '##ne', '##w', 'delhi', 'vs', '.', 'respond', '##ent', ':', 'm', '/', 's', '.', 'chun', '##i', 'lal', 'moon', '##ga', 'ram']
['date', 'of', 'judgment', ':', '05', '/', '05', '/', '1961', 'bench', ':', 'das', ',', 's', '.', 'k', '.']
['bench', ':', 'das', ',', 's', '.', 'k', '.', 'hid', '##aya', '##tu', '##llah', ',', 'm', '.', 'shah', ',', 'j', '.', 'c', '.', 'citation', ':', '1962', 'sc', '##r', '(', '2', ')', '82', '##3', '1962', 'air', '127', '##2', 'act', ':', 'excess', 'profits', 'tax', '-', 'income', '-', 'assess', '##ee', '##car', '##ry', '##ing', 'on', 'business', 'in', 'taxa', '##ble', 'territory', '-', 'losses', 'incurred', 'in', 'transactions', '##in', 'non', '-', 'taxa', '##ble', 'territory', '-', 'if', 'allow', '##able', 'in', 'computing', 'income', '-', 'excess', 'profits', 'tax', 'act', ',', '1940', '(', '15', 'of', '1940', ')', ',', 's', '.', '5', '.', 'head', '##note', ':', 'during', 'the', 'a

KeyboardInterrupt: 

In [150]:
len(sent2tensors(train_sents[0][0])['input_ids'].squeeze(0))

['petition', '##er', ':', 'the', 'commissioner', 'of', 'income', '-', 'tax', '##ne', '##w', 'delhi', 'vs', '.', 'respond', '##ent', ':', 'm', '/', 's', '.', 'chun', '##i', 'lal', 'moon', '##ga', 'ram']


512

In [153]:
result.size()

torch.Size([10, 768])